In [1]:
# This notebook is the only one that doesn't rely on analysis_variables for codes and data enrichment vars
# because the kinds of statistical tests/questions we want will be specific to the results of each analysis

In [4]:
import pandas as pd
from scipy.stats import ttest_ind, f_oneway
from utility_functions import load_file, starting_run, finished_run
from analysis_variables import logreg_targets, de_col_keys, de_col_values

In [3]:
dataset = load_file("summary_costs_enhanced.pickle").join(load_file("category_status_filtered.pickle"))
# logreg_targets = load_file("Full_logreg_targets.pickle")

In [7]:
def test(data, col_name_index, col_value_index):
    print(ttest_ind(
        data[data[de_col_keys[col_name_index]] == de_col_value[de_col_keys[col_name_index]][col_value_index]]["Cost"],
        data[data[de_col_keys[col_name_index]] != de_col_value[de_col_keys[col_name_index]][col_value_index]]["Cost"],
        equal_var = False,
#         alternative = "greater"
    ))

In [8]:
de_col_keys

['Admission Status', 'Biliary Colic Type', 'Cholecystectomy Type']

In [5]:
test(
    dataset
)

Ttest_indResult(statistic=13.336646639661922, pvalue=7.939789679811304e-40)


In [12]:
dataset.groupby(['Biliary Colic Type', 'Admission Status', 'Cholecystectomy Type'])[['Cost', 'ED Readmissions']].agg(['mean','count'])

Cost        \
                                                                  mean count   
Biliary Colic Type Admission Status Cholecystectomy Type                       
Complicated        Admitted         Delayed               11029.254941   166   
                                    Immediate             10754.346624  3607   
                                    No Surgery             9362.261106   876   
                                    Return Emergency      23305.676826    26   
                   Not Admitted     Delayed                4662.323950    32   
                                    No Surgery             1308.014732   127   
                                    Return Emergency       7763.618819    12   
Uncomplicated      Admitted         Delayed               10767.530688    56   
                                    Immediate             11834.135527   119   
                                    No Surgery             8876.249622   340   
                                    Return Emergency      17779.966747     7   
                   Not Admitted     Delayed                1854.780297    59   
                                    No Surgery              911.159282   234   
                                    Return Emergency       7402.488399    11   

                                                         ED Readmissions        
                                                                    mean count  
Biliary Colic Type Admission Status Cholecystectomy Type                        
Complicated        Admitted         Delayed                     0.030120   166  
                                    Immediate                   0.011644  3607  
                                    No Surgery                  0.035388   876  
                                    Return Emergency            1.038462    26  
                   Not Admitted     Delayed                     0.031250    32  
                                    No Surgery                  0.047244   127  
                                    Return Emergency            1.000000    12  
Uncomplicated      Admitted         Delayed                     0.053571    56  
                                    Immediate                   0.000000   119  
                                    No Surgery                  0.029412   340  
                                    Return Emergency            1.000000     7  
                   Not Admitted     Delayed                     0.084746    59  
                                    No Surgery                  0.029915   234  
                                    Return Emergency            1.000000    11

In [6]:
ttest_ind(
    dataset.loc[dataset["surgery_type"] == "Emergency Surgery"]["ED Readmissions"],
    dataset.loc[dataset["surgery_type"] != "Emergency Surgery"]["ED Readmissions"],
    equal_var = False,
    alternative = "less"
)

Ttest_indResult(statistic=-6.123143831957243, pvalue=5.757754126114558e-10)

In [7]:
dataset.groupby("surgery_type")["ED Readmissions"].agg(["sum", "count", "mean"])

,sum,count,mean
surgery_type,,,
Delayed Surgery,10.0,229,0.043668
ED Readmit Surgery,27.0,27,1.000000
Emergency Surgery,27.0,2709,0.009967
No Surgery,31.0,1096,0.028285


In [8]:
def compare_cost(target):
    data = dataset.join(
        logreg_targets[target](dataset).rename('comp'),
        how="inner"
    )
    p = ttest_ind(
        data[data["comp"] == False]["Cost"],
        data[data["comp"] == True]["Cost"],
        equal_var = False
    )
    print (f"p = {p[1]}")
    return (
        data.groupby('comp')['Cost'].mean()
    )
    

In [9]:
compare_cost("Admission vs Discharge")

p = 0.0


comp
False     1434.876420
True     10737.786787
Name: Cost, dtype: float64

In [10]:
compare_cost('Delayed + Delayed Emergency vs No Surgery')

p = 0.0009684376664753035


comp
False    7046.435888
True     9139.202212
Name: Cost, dtype: float64

In [11]:
compare_cost('Given Admission - Immediate Surgery vs non-Surgical Management')

p = 3.993031089877197e-05


comp
False     9817.308391
True     11067.038749
Name: Cost, dtype: float64

In [12]:
dataset.groupby("surgery_type")["Cost"].agg(["mean", "sem", "count"])

,mean,sem,count
surgery_type,,,
Delayed Surgery,8083.023816,489.415886,229
ED Readmit Surgery,18097.159722,3354.748917,27
Emergency Surgery,11067.038749,176.443120,2709
No Surgery,7046.435888,212.695959,1096


In [13]:
dataset.columns

Index(['died', 'initial_record_id', 'initial_year', 'age', 'homeless', 'race',
       'median_zip_income', 'payer', 'initial_discharge_quarter', 'admitted',
       'max_year', 'ED Readmissions', 'Surgery Visits',
       'Inpatient Readmissions', 'surgery_type', 'gender', 'marital_status',
       'age_groups', 'CPT Costs', 'SID_costs', 'Cost', 'systemic hypertension',
       'obesity', 'aspirin', 'nicotine dependence', 'hyperlipidemia',
       'diabetes mellitus', 'ischemic heart disease',
       'alcohol-related disorders', 'biliary colic with inflammation',
       'mood disorders'],
      dtype='object')

In [15]:
dataset.groupby(["surgery_type", "ED Readmissions", "Inpatient Readmissions"])["Cost"].agg(["mean", "sem", "count"])

mean  \
surgery_type       ED Readmissions Inpatient Readmissions                 
Delayed Surgery    0.0             0.0                      6269.188844   
                                   1.0                     19898.439747   
                                   2.0                     42868.666025   
                   1.0             0.0                      1641.581760   
                                   1.0                     14415.819707   
                                   2.0                     27830.844584   
                   2.0             1.0                      4896.302720   
ED Readmit Surgery 1.0             0.0                      3109.285452   
                                   1.0                     19970.644006   
Emergency Surgery  0.0             0.0                     10996.872343   
                                   1.0                     24232.414644   
                   1.0             0.0                      4338.221797   
                                   1.0                     19268.364147   
No Surgery         0.0             0.0                      6651.490095   
                                   1.0                     14879.614294   
                   1.0             0.0                       774.711392   
                                   1.0                     17431.597273   
                                   2.0                     13475.436574   
                   2.0             2.0                     28220.120812   

                                                                    sem  count  
surgery_type       ED Readmissions Inpatient Readmissions                       
Delayed Surgery    0.0             0.0                       347.878255    195  
                                   1.0                      1802.993906     24  
                                   2.0                              NaN      1  
                   1.0             0.0                       181.077988      2  
                                   1.0                      3520.274472      5  
                                   2.0                              NaN      1  
                   2.0             1.0                              NaN      1  
ED Readmit Surgery 1.0             0.0                       732.745867      3  
                                   1.0                      3596.288753     24  
Emergency Surgery  0.0             0.0                       176.139901   2680  
                                   1.0                     11812.974180      2  
                   1.0             0.0                      1261.226239      4  
                                   1.0                      2532.746348     23  
No Surgery         0.0             0.0                       202.033614   1051  
                                   1.0                      2159.957756     17  
                   1.0             0.0                        11.879868      2  
                                   1.0                      2513.091787     21  
                                   2.0                      1255.631059      2  
                   2.0             2.0                      7590.941008      3

In [17]:
filtered_dataset_codes = load_file("fully_filtered_codes.pickle")

In [20]:
filtered_dataset_codes["codes"].str.contains("K81").value_counts()

False    86106
True        16
Name: codes, dtype: int64